In [5]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, log_loss

In [6]:
def load_data(data_path: str) -> pd.DataFrame:
    df = pd.read_excel(data_path,
                       usecols=['발명의 명칭', '요약', '메인IPC2', '대표청구항'])
    df['input'] = df.apply(
        lambda row: f"{row['발명의 명칭']}\n\n{row['요약']}", axis=1)
    df = df.drop(columns=['발명의 명칭', '요약', '대표청구항'])
    df.columns = ['메인IPC2', '대표청구항']
    return df


# 훈련 데이터 로드
xlsx_path_train = 'dataset/DS학술제-모델링경진대회_Train.xlsx'
df_train = load_data(xlsx_path_train)

# 테스트 데이터 로드
xlsx_path_test = 'dataset/DS학술제-모델링경진대회_Valid.xlsx'
df_test = load_data(xlsx_path_test)

# 라벨 인코딩
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train['메인IPC2'])
y_test = label_encoder.transform(df_test['메인IPC2'])

# 텍스트 데이터 추출
text_train = df_train['대표청구항']
text_test = df_test['대표청구항']

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(text_train)
X_test = tfidf.transform(text_test)

In [7]:
# KNN 모델 생성 및 학습
k = 5  # k값은 필요에 따라 조정 가능
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [8]:
# 예측 및 평가
y_pred = knn.predict(X_test)
y_pred_proba = knn.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
test_loss = log_loss(y_test, y_pred_proba)

# 결과 출력
print(f"테스트 손실 {test_loss:.4f}, 정확도 {accuracy:.4f}\n")

# 분류 리포트 출력
class_names = label_encoder.classes_
print("분류 리포트:")
print(classification_report(y_test, y_pred, target_names=class_names, digits=4))

테스트 손실 2.3219, 정확도 0.7824

분류 리포트:
              precision    recall  f1-score   support

        G06F     0.5398    0.6162    0.5755        99
        G06Q     0.8550    0.8817    0.8681       448
        G16H     0.5556    0.2727    0.3659        55

    accuracy                         0.7824       602
   macro avg     0.6501    0.5902    0.6032       602
weighted avg     0.7758    0.7824    0.7741       602

